In [84]:
import requests
from bs4 import BeautifulSoup
import json
import re
import ast
import time

url_evo = "https://portal.rockgympro.com/portal/public/e758e0e391200c87c7bea411f033aab2/occupancy"
url_saltpump = "https://portal.rockgympro.com/portal/public/8490bc5e774d0034d09420df23a224b9/occupancy"

headers = {
    # 'Host': "portal.rockgympro.com",
    # 'Connection': "keep-alive",
    # 'Cache-Control': "max-age=0",
    # 'Upgrade-Insecure-Requests': "1",
    'User-Agent': "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.68 Mobile Safari/537.36",
    # 'Accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    # 'Sec-Fetch-Site': "none",
    # 'Sec-Fetch-Mode': "navigate",
    # 'Sec-Fetch-User': "?1",
    # 'Sec-Fetch-Dest': "document",
    # 'Accept-Encoding': "gzip, deflate, br",
    # 'Accept-Language': "en-US,en;q=0.9"
}

r = requests.get(url_evo, headers = headers)

soup = BeautifulSoup(r.text, 'html.parser')

In [85]:
s = soup.find('body').find('script').string
s

'\n\n    var data = {\n        \'AAA\' : { \n    \'capacity\' : 40,\n    \'count\' : 3,\n    \'subLabel\' : \'Current climber count\',\n    \'lastUpdate\' : \'Last updated:&nbsp7 mins ago (9:48 AM)\'\n},    };\n\n    function showGym(tag) {\n\n                tag=Object.keys(data)[0];        \n        \n        if (!tag) {\n            $(\'#main-content\').hide();\n            return;\n        } else {\n            $(\'#main-content\').show();\n        }\n        var dataElement = data[tag];\n        if (!dataElement) {\n            return;\n        }\n\n        var opts = {\n            angle: 0.10, // The span of the gauge arc\n            lineWidth: 0.44, // The line thickness\n            radiusScale: 1, // Relative radius\n            pointer: {\n                length: 0.6, // // Relative to gauge radius\n                strokeWidth: 0.035, // The thickness\n                color: \'black\' // Fill color\n            },\n            limitMax: false,     // If false, max value inc

In [98]:
d = ast.literal_eval(s.partition("var data = ")[2].partition(";")[0])
d

{'capacity': 40,
 'count': 3,
 'subLabel': 'Current climber count',
 'lastUpdate': 'Last updated:&nbsp7 mins ago (9:48 AM)'}

In [101]:
def scrape_counter(gym_id):

    headers = {
        'User-Agent': "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.68 Mobile Safari/537.36",
    }

    url = "https://portal.rockgympro.com/portal/public/" + gym_id + "/occupancy"

    r = requests.get(url, headers = headers)

    soup = BeautifulSoup(r.text, 'html.parser')

    data_script = soup.find('body').find('script').string
    data_dict = ast.literal_eval(s.partition("var data = ")[2].partition(";")[0])['AAA']

    out = {
        "gym_id": gym_id,
        "accessed_at": time.time(),
        "count": data_dict['count'],
        "capacity": data_dict['capacity']
    }

    return(out)


In [102]:
scrape_counter("e758e0e391200c87c7bea411f033aab2")

{'gym_id': 'e758e0e391200c87c7bea411f033aab2',
 'accessed_at': 1610032354.713402,
 'count': 3,
 'capacity': 40}

In [92]:
with open("gyms.json", "r") as read_file:
    data = json.load(read_file)

print(data)

[{'gym': 'Evo', 'city': 'Portland', 'state': 'ME', 'id': 'e758e0e391200c87c7bea411f033aab2'}, {'gym': 'Salt Pump', 'city': 'Scarborough', 'state': 'ME', 'id': '8490bc5e774d0034d09420df23a224b9'}]


In [103]:
results = []
for gym in data:
    current_scrape = scrape_counter(gym['id'])
    results.append(current_scrape)
    time.sleep(2)

In [ ]:
with open("gym_counter_results.json") as data_file:
    past_results = json.load(data_file)

results = past_results + results

with open('gym_counter_results.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)
    

In [105]:
results

[{'gym_id': 'e758e0e391200c87c7bea411f033aab2',
  'accessed_at': 1610032420.137992,
  'count': 3,
  'capacity': 40},
 {'gym_id': '8490bc5e774d0034d09420df23a224b9',
  'accessed_at': 1610032422.690337,
  'count': 3,
  'capacity': 40}]